# Dijkstra's Algorithm
In the "Greedy Algorithms" lesson, we implemented the **Dijkstra's Algorithm** to find the distance of each node from the given source node. In this exercise, you'll implement the same **Dijkstra's algorithm to find the length of the shortest path between a given pair of nodes,** but this time we will have a better time complexity. First, let's build the graph.

## Graph Representation
In order to run Dijkstra's Algorithm, we'll need to add distance to each edge. We'll use the `GraphEdge` class below to represent each edge between a pair of nodes. You are free to create your own implementation of an undirected graph. 

In [115]:
# Helper Class
class GraphEdge(object):
    def __init__(self, destinationNode, distance):
        self.node = destinationNode
        self.distance = distance

    def __repr__(self):
        return self.node.value

The new graph representation should look like this:

In [117]:
# Helper Classes
class GraphNode(object):
    def __init__(self, val):
        self.value = val
        self.edges = []

    def add_child(self, node, distance):
        self.edges.append(GraphEdge(node, distance))

    def remove_child(self, del_node):
        if del_node in self.edges:
            self.edges.remove(del_node)

    def __repr__(self):
        return self.value

class Graph(object):
    def __init__(self, node_list):
        self.nodes = node_list

    # adds an edge between node1 and node2 in both directions
    def add_edge(self, node1, node2, distance):
        if node1 in self.nodes and node2 in self.nodes:
            node1.add_child(node2, distance)
            node2.add_child(node1, distance)

    def remove_edge(self, node1, node2):
        if node1 in self.nodes and node2 in self.nodes:
            node1.remove_child(node2)
            node2.remove_child(node1)

    def __len__(self):
        return len(self.nodes)

### Exercise - Write the function definition here
Using what you've learned, implement Dijkstra's Algorithm 

In [307]:
class Heap:
    def __init__(self, initial_size=10):
        self.cbt = [None for _ in range(initial_size)]  # initialize arrays
        self.next_index = 0  # denotes next index where new element should go

    def enq(self, node):
        # insert element at the next index
        self.cbt[self.next_index] = node

        # heapify
        self._up_heapify()

        # increase index by 1
        self.next_index += 1

        # double the array and copy elements if next_index goes out of array bounds
        if self.next_index >= len(self.cbt):
            temp = self.cbt
            self.cbt = [None for _ in range(2 * len(self.cbt))]

            for index in range(self.next_index):
                self.cbt[index] = temp[index]

    def deq(self):
        if self.size() == 0:
            return None
        self.next_index -= 1

        to_remove = self.cbt[0]
        last_element = self.cbt[self.next_index]

        # place last element of the cbt at the root
        self.cbt[0] = last_element

        # we do not remove the elementm, rather we allow next `insert` operation to overwrite it
        self.cbt[self.next_index] = to_remove
        self._down_heapify()
        return to_remove

    def size(self):
        return self.next_index 

    def is_empty(self):
        return self.size() == 0

    def _up_heapify(self):
        child_index = self.next_index

        while child_index >= 1:
            parent_index = (child_index - 1) // 2
            parent_element = self.cbt[parent_index]
            child_element = self.cbt[child_index]

            if parent_element.distance > child_element.distance:
                self.cbt[parent_index] = child_element
                self.cbt[child_index] = parent_element

                child_index = parent_index
            else:
                break

    def _down_heapify(self):
        parent_index = 0

        while parent_index < self.next_index:
            left_child_index = 2 * parent_index + 1
            right_child_index = 2 * parent_index + 2

            parent = self.cbt[parent_index]
            left_child = None
            right_child = None

            min_element = parent

            # check if left child exists
            if left_child_index < self.next_index:
                left_child = self.cbt[left_child_index]

            # check if right child exists
            if right_child_index < self.next_index:
                right_child = self.cbt[right_child_index]

            # compare with left child
            if left_child is not None:
                if parent.distance < left_child.distance:
                    min_element = parent
                else:
                    min_element = left_child

            # compare with right child
            if right_child is not None:
                if right_child.distance < left_child.distance:
                    min_element = right_child
                else:
                    min_element = min_element

            # check if parent is rightly placed
            if min_element == parent:
                return

            if min_element == left_child:
                self.cbt[left_child_index] = parent
                self.cbt[parent_index] = min_element
                parent = left_child_index

            elif min_element == right_child:
                self.cbt[right_child_index] = parent
                self.cbt[parent_index] = min_element
                parent = right_child_index

    def get_minimum(self):
        # Returns the minimum element present in the heap
        if self.size() == 0:
            return None
        return self.cbt[0]

    def print(self):
        print('\nTOP OF QUEUE')
        for node in self.cbt:
            if node:
                print(node.node, node.distance)
        print('BOTTOM OF QUEUE')

In [324]:
## WITH ANNOTATIONS
import math
import sys

def dijkstra(graph, start_node, end_node):

    # Create distances dictionary to store shortest path distances for each node, setting source at 0 and others at infinity
    distances = {key : sys.maxsize for key in graph.nodes}
    distances[start_node] = 0
    
    # Create an unvisited set to keep track of nodes that have not been visited
    visited = set()
    explored = list()
    current_node = None
    q = Heap()

    # Repeat until all nodes have been visited:
    while len(visited) < len(graph):
        print('-'*100)

        # Set the current node as the node with smallest value
        if current_node is None:
            current_node = start_node
            explored.append(current_node)
        else:
            current_node = q.deq().node
        
        print(f'Current node: {current_node}')
        print(f'Current node edges: {current_node.edges}')

        # Find all edges of the current node
        for edge in current_node.edges:
            print(f'Current edge: {edge}')
       
            # Enqueue each edge
            if edge.node not in explored:
                q.enq(edge)
                explored.append(edge.node)
    
            # Update values in the distances dictionary
            distance = distances[current_node] + edge.distance
            print(f'The distance from {start_node} to {edge.node} is {distance}')
            if distance < distances[edge.node]:
                distances[edge.node] = distance

        print(f'\nDistances: {distances}')

        # Add the current node to the visited set
        visited.add(current_node)
        print(f'Visited: {visited}')
        
    # Return the distance for the end node
    return distances[end_node]

dijkstra(graph,node_u,node_y)

----------------------------------------------------------------------------------------------------
Current node: U
Current node edges: [A, C, D]
Current edge: A
The distance from U to A is 4
Current edge: C
The distance from U to C is 6
Current edge: D
The distance from U to D is 3

Distances: {U: 0, D: 3, A: 4, C: 6, I: 9223372036854775807, T: 9223372036854775807, Y: 9223372036854775807}
Visited: {U}
----------------------------------------------------------------------------------------------------
Current node: D
Current node edges: [U, C]
Current edge: U
The distance from U to U is 6
Current edge: C
The distance from U to C is 7

Distances: {U: 0, D: 3, A: 4, C: 6, I: 9223372036854775807, T: 9223372036854775807, Y: 9223372036854775807}
Visited: {D, U}
----------------------------------------------------------------------------------------------------
Current node: A
Current node edges: [U, I]
Current edge: U
The distance from U to U is 8
Current edge: I
The distance from U to I i

14

In [318]:
# CLEAN VERSION
import math
import sys

def dijkstra(graph, start_node, end_node):

    # Create distances dictionary to store shortest path distances for each node, setting source at 0 and others at infinity
    distances = {key : sys.maxsize for key in graph.nodes}
    distances[start_node] = 0
    
    # Create an unvisited set to keep track of nodes that have not been visited
    visited = set()
    explored = list()
    current_node = None
    q = Heap()

    # Repeat until all nodes have been visited:
    while len(visited) < len(graph):

        # Set the current node as the node with smallest value
        if current_node is None:
            current_node = start_node
            explored.append(current_node)
        else:
            current_node = q.deq().node

        # Find all edges of the current node
        for edge in current_node.edges:
       
            # Enqueue each edge that has not already been previously explored
            if edge.node not in explored:
                q.enq(edge)
                explored.append(edge.node)
    
            # Update values in the distances dictionary
            distance = distances[current_node] + edge.distance
            if distance < distances[edge.node]:
                distances[edge.node] = distance

        # Add the current node to the visited set
        visited.add(current_node)
        
    # Return the distance for the end node
    return distances[end_node]

dijkstra(graph,node_u,node_y)

14

<span class="graffiti-highlight graffiti-id_6vmf0hp-id_cjtybve"><i></i><button>Show Solution</button></span>

In [328]:
import math   # Need math.inf constant

def dijkstra(graph, start_node, end_node):
    # Create a dictionary that stores the distance to all nodes in the form of node:distance as key:value 
    # Assume the initial distance to all nodes is infinity.
    # Use math.inf as a predefined constant equal to positive infinity 
    distance_dict = {node: math.inf for node in graph.nodes}
    
    # Build a dictionary that will store the "shortest" distance to all nodes, wrt the start_node
    shortest_distance = {}

    distance_dict[start_node] = 0
    
    while distance_dict:
        
        # Sort the distance_dict, and pick the key:value having smallest distance
        current_node, node_distance = sorted(distance_dict.items(), key=lambda x: x[1])[0]
        
        # Remove the current node from the distance_dict, and store the same key:value in shortest_distance
        shortest_distance[current_node] = distance_dict.pop(current_node)

        # Check for each neighbour of current_node, if the distance_to_neighbour is smaller than the alreday stored distance, update the distance_dict
        for edge in current_node.edges:
            if edge.node in distance_dict:
                
                distance_to_neighbour = node_distance + edge.distance
                if distance_dict[edge.node] > distance_to_neighbour:
                    distance_dict[edge.node] = distance_to_neighbour
    
    return shortest_distance[end_node]

dijkstra(graph,node_u,node_y)

14

### Test - Let's test your function

In [317]:
# Test Case 1:

# Create a graph
node_u = GraphNode('U')
node_d = GraphNode('D')
node_a = GraphNode('A')
node_c = GraphNode('C')
node_i = GraphNode('I')
node_t = GraphNode('T')
node_y = GraphNode('Y')

graph = Graph([node_u, node_d, node_a, node_c, node_i, node_t, node_y])

# add_edge() function will add an edge between node1 and node2 in both directions
graph.add_edge(node_u, node_a, 4)
graph.add_edge(node_u, node_c, 6)
graph.add_edge(node_u, node_d, 3)
graph.add_edge(node_d, node_c, 4)
graph.add_edge(node_a, node_i, 7)
graph.add_edge(node_c, node_i, 4)
graph.add_edge(node_c, node_t, 5)
graph.add_edge(node_i, node_y, 4)
graph.add_edge(node_t, node_y, 5)

# Shortest Distance from U to Y is 14
print('Shortest Distance from {} to {} is {}'.format(node_u.value, node_y.value, dijkstra(graph, node_u, node_y)))

Shortest Distance from U to Y is 14


In [312]:
# Test Case 2
node_A = GraphNode('A')
node_B = GraphNode('B')
node_C = GraphNode('C')

graph = Graph([node_A, node_B, node_C])

graph.add_edge(node_A, node_B, 5)
graph.add_edge(node_B, node_C, 5)
graph.add_edge(node_A, node_C, 10)

# Shortest Distance from A to C is 10
print('Shortest Distance from {} to {} is {}'.format(node_A.value, node_C.value, dijkstra(graph, node_A, node_C)))

Shortest Distance from A to C is 10


In [313]:
# Test Case 3
node_A = GraphNode('A')
node_B = GraphNode('B')
node_C = GraphNode('C')
node_D = GraphNode('D')
node_E = GraphNode('E')

graph = Graph([node_A, node_B, node_C, node_D, node_E])

graph.add_edge(node_A, node_B, 3)
graph.add_edge(node_A, node_D, 2)
graph.add_edge(node_B, node_D, 4)
graph.add_edge(node_B, node_E, 6)
graph.add_edge(node_B, node_C, 1)
graph.add_edge(node_C, node_E, 2)
graph.add_edge(node_E, node_D, 1)

# Shortest Distance from A to C is 4
print('Shortest Distance from {} to {} is {}'.format(node_A.value, node_C.value, dijkstra(graph, node_A, node_C)))

Shortest Distance from A to C is 4
